## **Important Libraries and Data**

In [1]:
import warnings
warnings.filterwarnings('ignore')

#importing necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import sys
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


In [2]:
#reading data
df = pd.read_csv("NBA_Dataset_csv.csv")
df_=df.copy()

## **Navigating the Data**

In [3]:
df_.head()

,Team,Points_Scored,Weightlifting_Sessions_Average,Yoga_Sessions_Average,Laps_Run_Per_Practice_Average,Water_Intake,Players_Absent_For_Sessions
0,Lakers,242,23.0,25.0,30.0,5.0,4.0
1,Lakers,144,24.0,26.0,31.0,5.0,8.0
2,Lakers,156,24.0,27.0,31.0,5.0,7.0
3,Lakers,159,26.0,29.0,34.0,5.0,1.0
4,Lakers,106,27.0,NaN,34.0,5.0,6.0


**Data Dictionary**

* Team: name of the team
* Points_Scored: points scored by the team
* Weightlifting_Sessions_Average: weighlifting sessions on an average done by the team
* Yoga_Sessions_Average: yoga sessions on an average done by the team
* Laps_Run_Per_Practice_Average: laps run on an average by the team
* Water_Intake: total water intake
* Players_Absent_For_Sessions: number of players absent for sessions



In [4]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 7 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Team                            159 non-null    object 
 1   Points_Scored                   159 non-null    int64  
 2   Weightlifting_Sessions_Average  150 non-null    float64
 3   Yoga_Sessions_Average           139 non-null    float64
 4   Laps_Run_Per_Practice_Average   145 non-null    float64
 5   Water_Intake                    145 non-null    float64
 6   Players_Absent_For_Sessions     130 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 8.8+ KB


* We can see that the full dataset contains 159 row, one categorical variable which is the Team Name and 6 numerical variables.
* There are missing values within the variable which we will deal with in the next steps.

In [5]:
df_.describe([0.05, 0.25, 0.50, 0.75, 0.95, 0.99]).T


,count,mean,std,min,5%,25%,50%,75%,95%,99%,max
Points_Scored,159.0,119.327044,52.354983,0.0,10.0,96.5,125.0,151.5,197.30,245.36,250.0
Weightlifting_Sessions_Average,150.0,7433.120000,90719.720242,8.0,11.0,19.0,24.0,33.0,42.55,57.53,1111111.0
Yoga_Sessions_Average,139.0,21.690647,11.439728,3.0,8.9,13.0,19.0,28.0,45.10,60.00,63.0
Laps_Run_Per_Practice_Average,145.0,30.972414,11.979117,9.0,13.0,23.0,29.0,40.0,47.80,64.00,68.0
Water_Intake,145.0,3.648276,1.804820,1.0,1.0,2.0,3.0,5.0,7.00,7.56,8.0
Players_Absent_For_Sessions,130.0,4.415385,2.202129,1.0,1.0,3.0,4.0,6.0,8.00,8.00,8.0
